In [ ]:
%matplotlib inline 

import six
import math
import time
import Quandl
import calendar
import warnings
import numpy as np
import pandas as pd
import seaborn as sb
import pylab as pylab
from docx import Document
from datetime import datetime

from matplotlib import colors
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_toolkits.mplot3d import Axes3D

from scipy.interpolate import interp1d
import statsmodels.stats.stattools as stats_stattools
import statsmodels.tsa.stattools as tsa_stattools
import statsmodels.tsa.seasonal as tsa_seasonal
import statsmodels.api as sm 
import xgboost as xgb
from unbalanced_dataset import SMOTE

from sklearn import svm
from sklearn import metrics, cross_validation, linear_model, naive_bayes, neighbors, ensemble
from sklearn import feature_selection
from sklearn import decomposition
from sklearn import discriminant_analysis
from sklearn import preprocessing

import sys
from os import listdir
from os.path import isfile, join
from helpers import features_analysis, procces_stocks, data_manipulation, download_quandl_data, ml_dataset, classifier_utils, report_generator, Iteration, Stacking, Boosting

fig_size = [10, 6]
plt.rcParams["figure.figsize"] = fig_size
sb.set_style('darkgrid')
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
GOLD = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/GOLD.csv')
SILVER = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/SILVER.csv')
PLAT = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/PLAT.csv')
OIL_BRENT = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/OIL_BRENT.csv')

USD_GBP = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/USD_GBP.csv')
JPY_USD = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/JPY_USD.csv')
AUD_USD = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/AUD_USD.csv')

INDEX_DJIA = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_DJIA.csv')
INDEX_HSI = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_HSI.csv')
INDEX_IBEX = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_IBEX.csv')
INDEX_N225 = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_N225.csv')
INDEX_SP500 = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_SP500.csv')
INDEX_AXJO = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_AXJO.csv')
INDEX_FCHI = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_FCHI.csv')
INDEX_GDAXI = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_GDAXI.csv')

Santander = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/Santander.csv')


values_names = ['GOLD', 'SILVER', 'PLAT', 'OIL_BRENT', 'USD_GBP', 'JPY_USD', 'AUD_USD', 'DJIA', 'HSI', 'IBEX', 'N225', 'SP500', 'AXJO', 'FCHI', 'GDAXI']
values_dfs = [GOLD, SILVER, PLAT, OIL_BRENT, USD_GBP, JPY_USD, AUD_USD, INDEX_DJIA, INDEX_HSI, INDEX_IBEX, INDEX_N225, INDEX_SP500, INDEX_AXJO, INDEX_FCHI, INDEX_GDAXI]
values_cols = ['USD', 'Value', 'Open', 'Close', 'High', 'Low', 'Volume']
dict_dfs_cols = {}
dict_dfs_cols2 = {}

for index in range(len(values_names)):
    name = values_names[index]
    df = values_dfs[index]    
    cols = df.columns.values

    new_cols = [x for x in cols if x not in ['Date', 'USD', 'Value', 'Open', 'Close', 'High', 'Low', 'Volume']]    
    new_cols2 = [x for x in cols if x not in ['Date']]    

    dict_dfs_cols[name] = new_cols
    dict_dfs_cols2[name] = new_cols2

dataset = ml_dataset.generate_df_dataset(values_names, values_dfs, dict_dfs_cols)
dataset_all = ml_dataset.generate_df_dataset(values_names, values_dfs, dict_dfs_cols2)

#First 30 row
dataset = dataset[31:]
dataset = dataset.reset_index(drop=True)

dataset_all = dataset_all[31:]
dataset_all = dataset_all.reset_index(drop=True))  

In [ ]:
#training_dates = Iteration.Iteration('1995-08-18', '2004-01-23')
#testing_dates  = Iteration.Iteration('2004-01-26', '2004-04-21') #up
training_dates = Iteration.Iteration('2001-08-17', '2008-04-16')
testing_dates  = Iteration.Iteration('2008-04-17', '2008-07-14') #down
#training_dates = Iteration.Iteration('2005-08-19', '2011-04-15')
#testing_dates  = Iteration.Iteration('2011-04-18', '2011-07-14') #down
#training_dates = Iteration.Iteration('2009-08-19', '2016-01-22')
#testing_dates  = Iteration.Iteration('2016-01-25', '2016-04-20') #up

training_dates.calculate_indices(dataset)
testing_dates.calculate_indices(dataset)

trainDates = []
testDates = []
trainDates.append(training_dates.lowerIndex)
trainDates.append(training_dates.upperIndex)
testDates.append(testing_dates.lowerIndex)
testDates.append(testing_dates.upperIndex)

print testDates
print testDates[1] - testDates[0]

In [ ]:
####################################################################################################
#                                      Benchmark 1                                                 #
####################################################################################################

first_day = dataset_all.IBEX_Close.iloc[testDates[0]]
last_day = dataset_all.IBEX_Close.iloc[testDates[1]]
capital = 10000
stock_value = (last_day - first_day)
profit = (capital)*((last_day - first_day)/(first_day))
percentaje = (last_day - first_day)/(first_day)
print "Capital %d" % capital
print "Start %.2f End %.2f" % (first_day, last_day)
print "Stock difference: %s" % str(stock_value)
print "Tax: %s" % str(tax)
print "Profit: %s" % str(profit)
print "Relative difference: %.2f" % (percentaje)
print "Percetn Relative difference: %.2f %%" % (percentaje*100)

In [ ]:
####################################################################################################
#                                      Benchmark 2                                                 #
####################################################################################################

init_cap = 10000 
capital = 10000
invested = False
first_day = dataset_all.IBEX_Close.iloc[testDates[0]]
last_day = dataset_all.IBEX_Close.iloc[testDates[1]]
last_invested_day = 0
last_invested_value = 0

for today_index in range(testDates[0],testDates[1],1):
    yestearday = dataset_all.IBEX_Close.iloc[today_index-1]
    today = dataset_all.IBEX_Close.iloc[today_index]
    tomorrow = dataset_all.IBEX_Close.iloc[today_index+1]    

    if (today - yestearday) > 0: #Stock is predicted to raise
        if invested == False:
            invested = True
            last_invested_day = today_index
            last_invested_value = today            
    else: #Sell if we have
        if invested == True:
            invested = False
            profit = (capital)*((today - last_invested_value)/(last_invested_value))
            capital += profit

if invested == True:
    invested = False
    profit = (capital)*((today - last_invested_value)/(last_invested_value))
    capital += profit
    
first_day = dataset_all.IBEX_Close.iloc[testDates[0]]
last_day = dataset_all.IBEX_Close.iloc[testDates[1]]
stock_value = (last_day - first_day)
cap_diff = capital - init_cap
percentaje = (capital - init_cap)/(init_cap)

print "Start %.2f End %.2f" % (first_day, last_day)
print "Initial Capital %2.f \t Final %2.f" % (init_cap,capital)
print "Stock difference: %s" % str(stock_value)
print "Profit: %s" % str(cap_diff)
print "Relative difference: %.2f" % (percentaje)
print "Percent Relative difference: %.2f %%" % (percentaje*100)

In [ ]:
model = svm.LinearSVC()#naive_bayes.BernoulliNB()
colY = 'IBEX_RD_B5_Close'
regex = 'RD_B5_Close'
for colY, regex, diffDays in zip(['IBEX_RD_B1_Close', 'IBEX_RD_B2_Close', 'IBEX_RD_B3_Close', 'IBEX_RD_B5_Close', 'IBEX_RD_B10_Close', 'IBEX_RD_B20_Close', 'IBEX_RD_B30_Close', 'IBEX_RD_B50_Close', 'IBEX_RD_B100_Close'],
                                 ['RD_B1_Close', 'RD_B2_Close', 'RD_B3_Close', 'RD_B5_Close', 'RD_B1_Close', 'RD_B20_Close', 'RD_B30_Close', 'RD_B50_Close', 'RD_B100_Close'],
                                 [1,2,3,5,10,20,30,50,100]): 

    colY = 'IBEX_RD_B30_Close'
    regex = 'RD_B30_Close'

    print "##############################"
    print "## Predicintg %s , regex %s ##" % (colY, regex)
    models_score = []
     
    for model in [linear_model.LogisticRegression(),linear_model.SGDClassifier(), 
                  discriminant_analysis.LinearDiscriminantAnalysis(), discriminant_analysis.QuadraticDiscriminantAnalysis(),
                  svm.SVC(),
                  naive_bayes.GaussianNB(), naive_bayes.BernoulliNB(),
                  neighbors.KNeighborsClassifier(),
                  ensemble.AdaBoostClassifier(), ensemble.BaggingClassifier(), ensemble.GradientBoostingClassifier(), ensemble.RandomForestClassifier(), ensemble.ExtraTreesClassifier()]:
        print model.__class__.__name__


        colsToShift = 1


        df_x = dataset.filter(regex=(regex))
        print df_x.columns
        last_row = list(range(df_x.shape[0] - colsToShift, df_x.shape[0] ))
        df_x = df_x.drop(last_row, axis=0)
        df_x = df_x.drop(colY, axis=1)
        df_x = df_x.drop('DJIA_RD_B30_Close', axis=1)
        df_x = df_x.drop('SP500_RD_B30_Close', axis=1)
        df_x = df_x.drop('GDAXI_RD_B30_Close', axis=1)
        df_x = df_x.drop('FCHI_RD_B30_Close', axis=1)
        print df_x.columns
        df_y = dataset[colY].shift(-colsToShift)
        last_row = list(range(df_y.shape[0] - colsToShift, df_y.shape[0] ))
        df_y = df_y.drop(last_row, axis=0)

        total = (trainDates[1]-trainDates[0]) + (testDates[1]-testDates[0])
        tr = float(trainDates[1]-trainDates[0]) / total * 100.0
        te = float(testDates[1]-testDates[0]) / total * 100.0

        trainX, trainY, testX, testY = ml_dataset.train_arrays_experiments(df_x, df_y, trainDates, testDates)
        model.fit(trainX, trainY)
        y_pred = model.predict(testX)  
        print "Score %.2f " % metrics.accuracy_score(testY, y_pred)
        
        ####################################################################################################
        #                           Machine Learning Model                                                 #
        ####################################################################################################

        predictions = y_pred
        init_cap = 10000 
        capital = 10000
        invested = False
        shares = 0
        first_day = dataset_all.IBEX_Close.iloc[testDates[0]]
        last_day = dataset_all.IBEX_Close.iloc[testDates[1]]
        today_index = testDates[0]
        last_invested_day = 0
        last_invested_value = 0

        for index_pred in range(y_pred.shape[0]-1):

            pred = predictions[index_pred]
            yestearday = dataset_all.IBEX_Close.iloc[today_index-1]
            today = dataset_all.IBEX_Close.iloc[today_index]
            tomorrow = dataset_all.IBEX_Close.iloc[today_index+1]    

            #print "Date %s \t Close %f \t Bin %d" % (dataset_all.iloc[today_index-diffDays+1]['Date'], dataset_all.iloc[today_index-diffDays+1]['IBEX_Close'],dataset_all.iloc[today_index-diffDays+1]['IBEX_RD_B10_Close']) 
            #print "Date %s \t Close %f \t Bin %d" % (dataset_all.iloc[today_index]['Date'], dataset_all.iloc[today_index]['IBEX_Close'],dataset_all.iloc[today_index]['IBEX_RD_B10_Close'])
            #print "Date %s \t Close %f \t Bin %d" % (dataset_all.iloc[today_index+1]['Date'], dataset_all.iloc[today_index+1]['IBEX_Close'],dataset_all.iloc[today_index+1]['IBEX_RD_B10_Close'])


            if pred > 0: #Stock is predicted to raise
                if invested == False:
                    diff_days_ago_value = dataset_all.iloc[testing_dates.lowerIndex-diffDays-1+index_pred]['IBEX_Close']
                    invested = True
                    last_invested_day = today_index
                    last_invested_value = today

            else: #Sell if we have
                if invested == True:
                    invested = False
                    profit = (capital)*((today - last_invested_value)/(last_invested_value))
                    capital += profit

            today_index += 1

        if invested == True:
            profit = (capital)*((today - last_invested_value)/(last_invested_value))
            capital += profit
            
        first_day = dataset_all.IBEX_Close.iloc[testDates[0]]
        last_day = dataset_all.IBEX_Close.iloc[testDates[1]]
        stock_value = (last_day - first_day)
        cap_diff = capital - init_cap
        percentaje = (capital - init_cap)/(init_cap)

        print "Start %.2f End %.2f" % (first_day, last_day)
        print "Initial Capital %2.f \t Final %2.f" % (init_cap,capital)
        print "Stock difference: %s" % str(stock_value)
        print "Profit: %s" % str(cap_diff)
        print "Relative difference: %.2f" % (percentaje)
        print "Percent Relative difference: %.2f %%" % (percentaje*100)
        print "............................................"